In [1]:
import numpy as np
import scipy
from scipy.special import expit
from scipy.optimize.linesearch import scalar_search_wolfe2

In [2]:
from nose.tools import assert_almost_equal, ok_, eq_

In [11]:
A = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
b = np.array([1, 1, -1, 1])
regcoef = 0.5
m = len(b)

x = np.zeros(2)

In [12]:
class BaseSmoothOracle(object):
    """
    Base class for implementation of oracles.
    """
    def func(self, x):
        """
        Computes the value of function at point x.
        """
        raise NotImplementedError('Func oracle is not implemented.')

    def grad(self, x):
        """
        Computes the gradient at point x.
        """
        raise NotImplementedError('Grad oracle is not implemented.')
    
    def hess(self, x):
        """
        Computes the Hessian matrix at point x.
        """
        raise NotImplementedError('Hessian oracle is not implemented.')
    
    def func_directional(self, x, d, alpha):
        """
        Computes phi(alpha) = f(x + alpha*d).
        """
        return np.squeeze(self.func(x + alpha * d))

    def grad_directional(self, x, d, alpha):
        """
        Computes phi'(alpha) = (f(x + alpha*d))'_{alpha}
        """
        return np.squeeze(self.grad(x + alpha * d).dot(d))

In [13]:
class LogRegL2Oracle(BaseSmoothOracle):
    """
    Oracle for logistic regression with l2 regularization:
         func(x) = 1/m sum_i log(1 + exp(-b_i * a_i^T x)) + regcoef / 2 ||x||_2^2.

    Let A and b be parameters of the logistic regression (feature matrix
    and labels vector respectively).
    For user-friendly interface use create_log_reg_oracle()

    Parameters
    ----------
        matvec_Ax : function
            Computes matrix-vector product Ax, where x is a vector of size n.
        matvec_ATx : function of x
            Computes matrix-vector product A^Tx, where x is a vector of size m.
        matmat_ATsA : function
            Computes matrix-matrix-matrix product A^T * Diag(s) * A,
    """
    def __init__(self, matvec_Ax, matvec_ATx, matmat_ATsA, b, regcoef):
        self.matvec_Ax = matvec_Ax
        self.matvec_ATx = matvec_ATx
        self.matmat_ATsA = matmat_ATsA
        self.b = b
        self.m = len(b)
        self.regcoef = regcoef

    def func(self, x):
        func = np.logaddexp(0, -self.b * self.matvec_Ax(x)).mean() + self.regcoef * (x @ x) / 2
        return func

    def grad(self, x):
        expit_curr = expit(-self.b * self.matvec_Ax(x))
        grad = -self.matvec_ATx(self.b * expit_curr) / self.m + self.regcoef * x
        return grad

    def hess(self, x):
        expit_curr = expit(-self.b @ self.matvec_Ax(x))
        n = len(x)
        ones = np.array([1] * self.m)
        hess = expit_curr * (1 - expit_curr) * self.matmat_ATsA(ones) / self.m + self.regcoef * np.eye(n)
        return hess

In [14]:
def create_log_reg_oracle(A, b, regcoef, oracle_type='usual'):
    """
    Auxiliary function for creating logistic regression oracles.
        `oracle_type` must be either 'usual' or 'optimized'
    """
    matvec_Ax = lambda x: A @ x  # TODO: Implement
    matvec_ATx = lambda x: A.T @ x  # TODO: Implement

    def matmat_ATsA(s):
        # TODO: Implement
        return A.T @ np.diag(s) @ A

    if oracle_type == 'usual':
        oracle = LogRegL2Oracle
    elif oracle_type == 'optimized':
        oracle = LogRegL2OptimizedOracle
    else:
        raise 'Unknown oracle_type=%s' % oracle_type
    return oracle(matvec_Ax, matvec_ATx, matmat_ATsA, b, regcoef)

In [15]:
A = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
b = np.array([1, 1, -1, 1])
reg_coef = 0.5

# Logistic regression oracle:
logreg = create_log_reg_oracle(A, b, reg_coef, oracle_type='usual')

# Check at point x = [0, 0]
x = np.zeros(2)
assert_almost_equal(logreg.func(x), 0.693147180)
ok_(np.allclose(logreg.grad(x), [0, -0.25]))
ok_(np.allclose(logreg.hess(x), [[0.625, 0.0625], [0.0625, 0.625]]))
ok_(isinstance(logreg.grad(x), np.ndarray))
ok_(isinstance(logreg.hess(x), np.ndarray))

# Check func_direction and grad_direction oracles at
# x = [0, 0], d = [1, 1], alpha = 0.5 and 1.0
x = np.zeros(2)
d = np.ones(2)
assert_almost_equal(logreg.func_directional(x, d, alpha=0.5),
                    0.7386407091095)
assert_almost_equal(logreg.grad_directional(x, d, alpha=0.5),
                    0.4267589549159)
assert_almost_equal(logreg.func_directional(x, d, alpha=1.0),
                    1.1116496416598)
assert_almost_equal(logreg.grad_directional(x, d, alpha=1.0),
                    1.0559278283039)

In [16]:
logreg.func_directional(x, d, alpha=0.5)

0.7386407091095953

In [17]:
alpha = 0.5
x = x + alpha * d

In [18]:
logreg.func([0.5, 0.5])

TypeError: unsupported operand type(s) for @: 'list' and 'list'

In [19]:
def func(x):
    func = np.logaddexp(0, -b * matvec_Ax(x)).mean() + regcoef * np.linalg.norm(x) / 2
    return func

In [21]:
matvec_Ax = lambda x: A @ x  # TODO: Implement
matvec_ATx = lambda x: A.T @ x  # TODO: Implement

def matmat_ATsA(s):
    # TODO: Implement
    return A.T @ np.diag(s) @ A

In [22]:
func([0.5, 0.5])

0.7904174044062322

In [23]:
0.7386407091095 - regcoef * np.linalg.norm(x) / 2

0.5618640138128631

In [24]:
-b * matvec_Ax(x)

array([-0. , -0.5,  0.5, -1. ])

In [25]:
class QuadraticOracle(BaseSmoothOracle):
    """
    Oracle for quadratic function:
       func(x) = 1/2 x^TAx - b^Tx.
    """

    def __init__(self, A, b):
        if not scipy.sparse.isspmatrix_dia(A) and not np.allclose(A, A.T):
            raise ValueError('A should be a symmetric matrix.')
        self.A = A
        self.b = b

    def func(self, x):
        return 0.5 * np.dot(self.A.dot(x), x) - self.b.dot(x)

    def grad(self, x):
        return self.A.dot(x) - self.b

    def hess(self, x):
        return self.A 

In [26]:
def grad_finite_diff(func, x, eps=1e-8):
    """
    Returns approximation of the gradient using finite differences:
        result_i := (f(x + eps * e_i) - f(x)) / eps,
        where e_i are coordinate vectors:
        e_i = (0, 0, ..., 0, 1, 0, ..., 0)
                          >> i <<
    """
    # TODO: Implement numerical estimation of the gradient
    n = len(x)
    grad_finite_diff = np.array([0.] * n)
    print(func(x))
    for i in range(n):
        e_i = np.zeros(n)
        e_i[i] = 1
        print('i', (func(x + eps * e_i) - func(x)) / eps)
        grad_finite_diff[i] = (func(x + eps * e_i) - func(x)) / eps
    return grad_finite_diff

In [27]:
A = np.eye(3)
b = np.array([1, 2, 3])
quadratic = QuadraticOracle(A, b)
g = grad_finite_diff(quadratic.func, np.zeros(3))
print(g)
ok_(isinstance(g, np.ndarray))
ok_(np.allclose(g, -b))

0.0
i -0.999999995
i -1.9999999949999998
i -2.9999999950000005
[-1.         -1.99999999 -3.        ]


In [28]:
def get_quadratic():
    # Quadratic function:
    #   f(x) = 1/2 x^T x - [1, 2, 3]^T x
    A = np.eye(3)
    b = np.array([1, 2, 3])
    return QuadraticOracle(A, b)

In [46]:
class LineSearchTool(object):
    """
    Line search tool for adaptively tuning the step size of the algorithm.

    method : String containing 'Wolfe', 'Armijo' or 'Constant'
        Method of tuning step-size.
        Must be be one of the following strings:
            - 'Wolfe' -- enforce strong Wolfe conditions;
            - 'Armijo" -- adaptive Armijo rule;
            - 'Constant' -- constant step size.
    kwargs :
        Additional parameters of line_search method:

        If method == 'Wolfe':
            c1, c2 : Constants for strong Wolfe conditions
            alpha_0 : Starting point for the backtracking procedure
                to be used in Armijo method in case of failure of Wolfe method.
        If method == 'Armijo':
            c1 : Constant for Armijo rule
            alpha_0 : Starting point for the backtracking procedure.
        If method == 'Constant':
            c : The step size which is returned on every step.
    """
    def __init__(self, method='Wolfe', **kwargs):
        self._method = method
        if self._method == 'Wolfe':
            self.c1 = kwargs.get('c1', 1e-4)
            self.c2 = kwargs.get('c2', 0.9)
            self.alpha_0 = kwargs.get('alpha_0', 1.0)
        elif self._method == 'Armijo':
            self.c1 = kwargs.get('c1', 1e-4)
            self.alpha_0 = kwargs.get('alpha_0', 1.0)
        elif self._method == 'Constant':
            self.c = kwargs.get('c', 1.0)
        else:
            raise ValueError('Unknown method {}'.format(method))

    @classmethod
    def from_dict(cls, options):
        if type(options) != dict:
            raise TypeError('LineSearchTool initializer must be of type dict')
        return cls(**options)

    def to_dict(self):
        return self.__dict__

    def line_search(self, oracle, x_k, d_k, previous_alpha=None):
        """
        Finds the step size alpha for a given starting point x_k
        and for a given search direction d_k that satisfies necessary
        conditions for phi(alpha) = oracle.func(x_k + alpha * d_k).

        Parameters
        ----------
        oracle : BaseSmoothOracle-descendant object
            Oracle with .func_directional() and .grad_directional() methods implemented for computing
            function values and its directional derivatives.
        x_k : np.array
            Starting point
        d_k : np.array
            Search direction
        previous_alpha : float or None
            Starting point to use instead of self.alpha_0 to keep the progress from
             previous steps. If None, self.alpha_0, is used as a starting point.

        Returns
        -------
        alpha : float or None if failure
            Chosen step size
        """
        # TODO: Implement line search procedures for Armijo, Wolfe and Constant steps.
        if self._method == 'Constant':
            return self.c
        alpha_0 = previous_alpha if previous_alpha else self.alpha_0
        phi = lambda a: oracle.func_directional(x_k, d_k, a)
        derphi = lambda a: oracle.grad_directional(x_k, d_k, a)
        c2 = self.c2 if self._method == 'Wolfe' else 0
        print(alpha_0)
        alpha, _, _, _ = scalar_search_wolfe2(phi=phi,
                                              derphi=derphi,
                                              phi0=phi(0),
                                              derphi0=derphi(0),
                                              c1=self.c1,
                                              c2=c2,  amax=alpha_0)
        print(alpha)
        if alpha is None:
            alpha = alpha_0
            while phi(alpha) > phi(0) + self.c1 * alpha * derphi(0):
                alpha /= 2
        print(alpha)
        return alpha

In [47]:
oracle = get_quadratic()
x = np.array([100, 0, 0])
d = np.array([-1, 0, 0])

In [48]:
# Constant line search
ls_tool = LineSearchTool(method='Constant', c=1.0)
assert_almost_equal(ls_tool.line_search(oracle, x, d, ), 1.0)
ls_tool = LineSearchTool(method='Constant', c=10.0)
assert_almost_equal(ls_tool.line_search(oracle, x, d), 10.0)

In [49]:
# Armijo rule
ls_tool = LineSearchTool(method='Armijo', alpha_0=100.0, c1=0.9)
assert_almost_equal(ls_tool.line_search(oracle, x, d), 12.5)

100.0
None
12.5


In [50]:
ls_tool = LineSearchTool(method='Armijo', alpha_0=100, c1=0.9)
assert_almost_equal(ls_tool.line_search(oracle, x, d, previous_alpha=1.0), 1.0)

ls_tool = LineSearchTool(method='Armijo', alpha_0=100, c1=0.95)
assert_almost_equal(ls_tool.line_search(oracle, x, d), 6.25)
ls_tool = LineSearchTool(method='Armijo', alpha_0=10, c1=0.9)
assert_almost_equal(ls_tool.line_search(oracle, x, d), 10.0)

1.0
None
1.0
100
None
6.25
10
None
10


/usr/local/lib/python3.6/dist-packages/scipy/optimize/linesearch.py:421: LineSearchWarning: The line search algorithm could not find a solution less than or equal to amax: 1.0
  warn(msg, LineSearchWarning)
/usr/local/lib/python3.6/dist-packages/scipy/optimize/linesearch.py:421: LineSearchWarning: The line search algorithm could not find a solution less than or equal to amax: 10
  warn(msg, LineSearchWarning)


In [51]:
np.ones(3) * 10.0

array([10., 10., 10.])

In [52]:
def get_line_search_tool(line_search_options=None):
    if line_search_options:
        if type(line_search_options) is LineSearchTool:
            return line_search_options
        else:
            return LineSearchTool.from_dict(line_search_options)
    else:
        return LineSearchTool()

In [53]:
oracle = get_quadratic()
x0 = np.ones(3) * 10.0
d0 = np.array([9., 8., 7.])

In [54]:
line_search_tool = get_line_search_tool(None)

In [55]:
line_search_tool.line_search(oracle, x0, d0)

1.0
None
5.551115123125783e-17


5.551115123125783e-17

In [60]:
oracle.grad_directional(x0, d0, 0)

194.0

In [61]:
oracle.func(x0) + 1e-4 * 194

90.0194

In [62]:
oracle.func([1, 2, 3])

-7.0

In [64]:
dic = {
    'a' : 1,
    'b' : 2
}

In [65]:
'a' in dic

True

In [69]:
import scipy.linalg as sla

In [70]:
A = np.array([
    [-1, 1],
    [1, -1]
])

In [ ]:
sla.cho_solve(sla.cho_factor(A), b)

In [71]:
sla.cho_factor(A)

LinAlgError: 1-th leading minor of the array is not positive definite

In [77]:
A = np.array([
    [1, 1],
    [1, 2]
])
b = np.array([1, 3])

In [75]:
sla.cho_factor(A)

(array([[1., 1.],
        [1., 1.]]), False)

In [78]:
sla.cho_solve(sla.cho_factor(A), b)

array([-1.,  2.])

In [79]:
x = np.array([10.0])

In [80]:
np.array([1 / (np.power(x, 2) + 1)])

array([[0.00990099]])

In [81]:
A = np.array([[0.]])

In [82]:
sla.cho_factor(A)

LinAlgError: 1-th leading minor of the array is not positive definite

In [84]:
np.random.uniform(size=2)

array([0.03574654, 0.43006079])

In [ ]:
np.random.uniform()